In [ ]:
# ==============================================================================
# CÉLULA ÚNICA FINAL PARA DataAnalysis2.ipynb
# ==============================================================================

import pandas as pd
from pathlib import Path
import sys
import matplotlib.pyplot as plt

# --- 1. SETUP E INICIALIZAÇÃO ---
print("--- 1. INICIANDO SETUP ---")
code_dir = Path('..').resolve() / 'Code'
if str(code_dir) not in sys.path:
    sys.path.append(str(code_dir))

from DataUtils import AnalisadorBancario

pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100) # Para ver mais nomes de bancos
plt.style.use('seaborn-v0_8-whitegrid')

output_dir = Path('..').resolve() / 'Output'
analisador = AnalisadorBancario(diretorio_output=str(output_dir))
print("-" * 50)

In [ ]:
# ==============================================================================
# CÉLULA 2: Gerar Arquivo de Referência com Nomes para Consulta
# ==============================================================================
print("\n--- 2. GERANDO ARQUIVO DE REFERÊNCIA COM NOMES DISPONÍVEIS ---")

# Define o caminho do arquivo Excel de saída
excel_output_path = Path('.') / 'referencia_nomes_consulta.xlsx'

# Usando um ExcelWriter para salvar múltiplas abas no mesmo arquivo
with pd.ExcelWriter(excel_output_path, engine='openpyxl') as writer:
    
    # 1. Nomes de Bancos
    print("Processando nomes de bancos...")
    # Pega a lista completa de nomes únicos e salva em uma aba
    nomes_bancos_df = analisador._mapa_nomes_df[['nome']].rename(columns={'nome': 'Nomes de Bancos Disponíveis'})
    nomes_bancos_df.to_excel(writer, sheet_name='Bancos', index=False)
    print(f"-> {len(nomes_bancos_df)} nomes de bancos salvos na aba 'Bancos'.")

    # 2. Nomes de Contas COSIF
    print("\nProcessando nomes de contas COSIF...")
    # Pega a lista completa, remove nulos, ordena e salva em outra aba
    contas_cosif_df = pd.DataFrame(
        analisador.df_cosif_prud['NOME_CONTA_COSIF'].dropna().unique(),
        columns=['Nome Conta COSIF']
    ).sort_values('Nome Conta COSIF').reset_index(drop=True)
    contas_cosif_df.to_excel(writer, sheet_name='Contas_COSIF', index=False)
    print(f"-> {len(contas_cosif_df)} nomes de contas COSIF salvos na aba 'Contas_COSIF'.")

    # 3. Nomes de Contas IFDATA
    print("\nProcessando nomes de contas IFDATA...")
    # Pega a lista completa, remove nulos, ordena e salva na terceira aba
    contas_ifdata_df = pd.DataFrame(
        analisador.df_ifd_val['NOME_CONTA_IFD_VAL'].dropna().unique(),
        columns=['Nome Conta IFDATA']
    ).sort_values('Nome Conta IFDATA').reset_index(drop=True)
    contas_ifdata_df.to_excel(writer, sheet_name='Contas_IFDATA', index=False)
    print(f"-> {len(contas_ifdata_df)} nomes de contas IFDATA salvos na aba 'Contas_IFDATA'.")

print(f"\nArquivo de referência salvo com sucesso em: '{excel_output_path.resolve()}'")
print("-" * 50)

In [ ]:
# --- 3. TESTE DE CONSULTA BÁSICA (COSIF) ---
print("\n--- 3. TESTANDO CONSULTA BÁSICA COSIF ---")
# Use um nome exato da lista acima para garantir o sucesso
dados_bb_cosif = analisador.get_dados_cosif(
    identificador='BCO DO BRASIL S.A.',
    contas=['ATIVO REALIZÁVEL', 'Caixa'],
    datas=202403
)
print("Resultado para 'BCO DO BRASIL S.A.':")
display(dados_bb_cosif)
print("-" * 50)


# --- 4. TESTE DA TABELA COMPARATIVA COMPLETA ---
print("\n--- 4. TESTANDO TABELA COMPARATIVA COMPLETA ---")
indicadores_completos = {
    'Patrimônio Líquido': {'tipo': 'COSIF', 'conta': 'PATRIMÔNIO LÍQUIDO'},
    'Total Ativos': {'tipo': 'COSIF', 'conta': 'TOTAL GERAL DO ATIVO'},
    'Índice de Basileia': {'tipo': 'IFDATA', 'conta': 'Índice de Basileia'},
    'Situação': {'tipo': 'Atributo', 'atributo': 'SITUACAO_IFD_CAD'}
}

# Usando nomes curtos, mas agora sabendo que a busca flexível funciona
bancos_para_comparar = ['ITAÚ UNIBANCO S.A.', 'BANCO BRADESCO', 'NU FINANCEIRA', 'BANCO BTG PACTUAL', 'BCO DO BRASIL S.A.']

print("\nComparativo usando Documento 4060 (Balancete):")
tabela_balancete = analisador.comparar_indicadores(
    identificadores=bancos_para_comparar,
    indicadores=indicadores_completos,
    data=202403,
    documento_cosif=4060 
)
display(tabela_balancete)
print("-" * 50)


# --- 5. TESTE DA SÉRIE TEMPORAL ---
print("\n--- 5. TESTANDO SÉRIE TEMPORAL ---")
serie_nubank = analisador.get_serie_temporal_indicador(
    identificador='NU FINANCEIRA', # Usando nome curto
    conta='TOTAL GERAL DO ATIVO',
    data_inicio=202401,
    data_fim=202412,
    fonte='COSIF',
    documento_cosif=4060
)

print("\nSérie Temporal para 'TOTAL GERAL DO ATIVO' do Nubank:")
display(serie_nubank)

if not serie_nubank.empty:
    fig, ax = plt.subplots(figsize=(12, 6))
    serie_nubank.plot(ax=ax, marker='o', linestyle='-')
    ax.set_title("Evolução do Ativo Total - Nubank (COSIF Prudencial Doc 4060)", fontsize=16)
    ax.set_ylabel("Valor (em R$)")
    ax.set_xlabel("Data")
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))
    plt.tight_layout()
    plt.show()

print("\n--- FIM DOS TESTES ---")